In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
import torch.nn as nn
import torch
from imblearn.over_sampling import SMOTE
from tqdm.auto import tqdm
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from torch.nn import functional as F
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torch import optim
plt.rcParams['font.family'] = 'Malgun Gothic'
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as sk
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import accuracy_score
import numpy as np
%matplotlib inline

In [26]:
df=pd.read_csv("../../data/군집화 라벨링_7days.csv",encoding='cp949')

In [27]:
df.head()

,ID,collect_datetime,gender,grade,height,weight,step_count,burned calory,eat_calory,Sleep_time,before_height,before_weight,before_waist,after_height,after_weight,after_waist,bmi,waist_bmi,class_num,labels
0,#AAGQKY,2022-07-21,2.0,4.0,148.0,45.15,72.0,8.877,2000.0,12.0,148.0,45.3,-,148.0,45.0,NaN,20.612673,0.306081,2,0
1,#AAGQKY,2022-07-22,2.0,4.0,148.0,45.15,72.0,8.877,2000.0,12.0,148.0,45.3,-,148.0,45.0,NaN,20.612673,0.306081,2,0
2,#AAGQKY,2022-07-23,2.0,4.0,148.0,45.15,72.0,8.877,2000.0,12.0,148.0,45.3,-,148.0,45.0,NaN,20.612673,0.306081,2,0
3,#AAGQKY,2022-07-24,2.0,4.0,148.0,45.15,72.0,8.877,2000.0,12.0,148.0,45.3,-,148.0,45.0,NaN,20.612673,0.306081,2,0
4,#AAGQKY,2022-07-25,2.0,4.0,148.0,45.15,72.0,8.877,2000.0,12.0,148.0,45.3,-,148.0,45.0,NaN,20.612673,0.306081,2,0


In [28]:
name_list=[]

In [29]:
for name in df['ID'].unique():
    df_ID=df[df["ID"]==name]
    if len(df_ID['labels'].unique())==3:
        name_list.append(name)
        

In [30]:
len(name_list)

113

In [31]:
len(df['ID'].unique())

327

In [32]:
name='#ASELLU'

In [33]:
df=df[df["ID"]==name]

In [34]:
len(df['labels'].unique())

3

In [35]:
df_data=df[['height','weight','step_count','burned calory','eat_calory','Sleep_time','labels']]
title='6 feature prediction'
term_np=np.array([])
MAE_np=np.array([])
MSE_np=np.array([])
RMSE_np=np.array([])
MAPE_np=np.array([])

In [36]:
CFG={'TRAIN_WINDOW_SIZE':5,
   'PREDICT_SIZE':1 ,
    'how_inputdim':6,
    'hidden_dim':10,
    'LEARNING_RATE':0.001,
    'nb_epochs':1,
     'target':3,
    'BATCH_SIZE':25}
term="{}train_size-{}predict_size".format(CFG['TRAIN_WINDOW_SIZE'],CFG['PREDICT_SIZE'])
print(term)

5train_size-1predict_size


# 데이터를 역순으로 정렬하여 전체 데이터의 70% 학습, 30% 테스트에 사용

X=df_data.iloc[:,:-1]
y=df_data.iloc[:,-1]


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)
smote = SMOTE(random_state=0)
X_over,y_over = smote.fit_resample(X_scaler,y)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_scaler.shape, y.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_over.shape, y_over.shape)
print('SMOTE 적용 전 레이블 값 분포: \n', pd.Series(y).value_counts())
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_over).value_counts())
y_over=y_over.to_numpy()
df_data_np = np.concatenate((X_over, y_over.reshape(-1, 1)), axis=1)
df_data = pd.DataFrame(df_data_np)

In [38]:
df_data['labels'].value_counts()

0    118
2     36
1     21
Name: labels, dtype: int64

In [39]:
train_size = int(len(df_data)*0.8)
train_set = df_data[0:train_size]  
test_set = df_data[train_size-CFG['TRAIN_WINDOW_SIZE']:]

# Input scale
scaler_x = StandardScaler()
scaler_x.fit(train_set.iloc[:, :-1])

train_set.iloc[:, :-1] = scaler_x.transform(train_set.iloc[:, :-1])
test_set.iloc[:, :-1] = scaler_x.transform(test_set.iloc[:, :-1])


# Output scale

train_set.iloc[:, -1:] = train_set.iloc[:, -1:]
test_set.iloc[:, -1:] = test_set.iloc[:, -1:]

# 데이터셋 생성 함수
def build_dataset(time_series,train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    dataX = []
    dataY = []
    window_size = train_size + predict_size
    for i in range(len(time_series) - window_size + 1):
        _x = time_series[i:i+train_size, :-1]
        _y = time_series[i+train_size:i+train_size+predict_size, [-1]]
        # print(_x, "-->",_y)
        dataX.append(_x)
        dataY.append(_y)

    return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(np.array(train_set))
testX, testY = build_dataset(np.array(test_set))

class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])

    def __len__(self):
        return len(self.X)

train_dataset = CustomDataset(trainX, trainY)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(testX, testY)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# test_dataset = CustomDataset(test_input, None)
# test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# 모델선언

C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\pandas\core\indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\pandas\core\indexing.py:1851: SettingWithCopyWarning: 
A value is trying to be set on 

In [40]:
# 모델선언
class BaseModel(nn.Module):
    def __init__(self, input_size=CFG['how_inputdim'], hidden_size=CFG['hidden_dim'], output_size=CFG['target']):
        super(BaseModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size, output_size)
        )

        self.actv = nn.ReLU()

    def forward(self, x):
        # x shape: (B, TRAIN_WINDOW_SIZE, 5)
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        # LSTM layer
        rnn_out, hidden = self.rnn(x, hidden)

        # Only use the last output sequence
        rnn_output = rnn_out[:, -1, :]
        # Fully connected layer
        output = self.fc(rnn_output)
        output=self.actv(output)
        return output

    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return (torch.zeros(1, batch_size, self.hidden_size, device=device),
                torch.zeros(1, batch_size, self.hidden_size, device=device))

In [41]:
# LSTM모델학습

def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None

    for epoch in range(1, CFG['nb_epochs']+1):
        model.train()
        train_loss = []
        train_mae = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)
            optimizer.zero_grad()

            output = model(X)
            ohe_label=F.one_hot(Y.long(), CFG['target'])

            loss = criterion(output, ohe_label.float())
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')
    return best_model

def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []

    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)

            output = model(X)
            ohe_label=F.one_hot(Y.long(), CFG['target'])

            loss = criterion(output, ohe_label.float())
            val_loss.append(loss.item())
    return np.mean(val_loss)

In [42]:
model = BaseModel()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, train_loader, val_loader, device)

def inference(model, test_loader, device):
    model.eval()
    predictions=[]
    with torch.no_grad():
        for X, Y in tqdm(iter(test_loader)):
            X = X.to(device)
            Y = Y.to(device)

            output = model(X)
           # output = output.cpu().numpy()
            preds = F.log_softmax(output, dim=1).argmax(dim=1)
            preds = preds.cpu().numpy()
            predictions.extend(preds)
            #print("predictions",predictions.shape)
            
    return np.array(predictions)

  0%|          | 0/6 [00:00<?, ?it/s]

C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([25, 1, 1, 3])) that is different to the input size (torch.Size([25, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([10, 1, 1, 3])) that is different to the input size (torch.Size([10, 3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.22410] Val Loss : [0.22989]
Model Saved


In [43]:
#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(4)]

In [44]:
Result

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [45]:
Result_df=pd.DataFrame(Result,columns=['Accuracy','F1-Score','Precision','Recall'])
Result_df

,Accuracy,F1-Score,Precision,Recall
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0


In [46]:
answer_np = np.empty((0,CFG['PREDICT_SIZE']))
for val in val_loader:
    answer_np = np.vstack((answer_np, val[1].view(-1,CFG['PREDICT_SIZE'])))
    

pred = inference(infer_model,val_loader , device)
#answer.extend(answer_np)
print("pred.shape",pred.shape)
print("pred",pred)

print("answer.shape",answer_np.shape)
print("answer",answer_np)
# 눤래 정답을 inverse scaling


accuracy=accuracy_score(answer_np.squeeze(), pred) * 100
print("Accuracy : {}".format(accuracy))   
#f1score
f1 = f1_score(answer_np.squeeze(),pred, average='weighted')
print("F1score : {}".format(f1))
#precision/recall
p_rlist=sk(answer_np.squeeze(),pred,average='weighted')
print("Precision : {}".format(p_rlist[0]))
print("Recall : {}".format(p_rlist[1]))
print()
 #결과 배열에 넣기
Result[0]=accuracy
Result[1]=f1
Result[2]=p_rlist[0]
Result[3]=p_rlist[1]

  0%|          | 0/2 [00:00<?, ?it/s]

pred.shape (35,)
pred [2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
answer.shape (35, 1)
answer [[2.]
 [2.]
 [2.]
 [2.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Accuracy : 62.857142857142854
F1score : 0.5024409763905562
Precision : 0.43660714285714286
Recall : 0.6285714285714286



C:\Users\jaemin\Anaconda3\envs\jm\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
Result

[0.0, 0.0, 0.0, 0.0]

In [50]:
np.unique(answer_np,return_counts=True)

(array([0., 1., 2.]), array([19, 12,  4], dtype=int64))

In [24]:
answer_np.squeeze()

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2.])

In [51]:
np.unique(pred,return_counts=True)

(array([0, 2], dtype=int64), array([32,  3], dtype=int64))